## Описание проекта

Из открытых источников доступны исторические данные о продажах игр интернет-магазина «Стримчик», который продаёт по всему миру компьютерные игры,
оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation).
Нужно выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.

Перед вами данные до 2016 года. Представим, что сейчас декабрь 2016 г., и вы планируете кампанию на 2017-й. Нужно отработать принцип работы с данными.
Неважно, прогнозируете ли вы продажи на 2017 год по данным 2016-го или же 2027-й — по данным 2026 года.
В наборе данных попадается аббревиатура ESRB (Entertainment Software Rating Board) — это ассоциация, определяющая возрастной рейтинг компьютерных игр.
ESRB оценивает игровой контент и присваивает ему подходящую возрастную категорию, например, «Для взрослых», «Для детей младшего возраста» или «Для подростков».


## Цель проекта

Выявить определяющие успешность игры закономерности, с целью сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.

### Описание данных

•	Name — название игры
•	Platform — платформа
•	Year_of_Release — год выпуска
•	Genre — жанр игры
•	NA_sales — продажи в Северной Америке (миллионы проданных копий)
•	EU_sales — продажи в Европе (миллионы проданных копий)
•	JP_sales — продажи в Японии (миллионы проданных копий)
•	Other_sales — продажи в других странах (миллионы проданных копий)
•	Critic_Score — оценка критиков (максимум 100)
•	User_Score — оценка пользователей (максимум 10)
•	Rating — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.


In [23]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objs as go
import plotly.express as px
from scipy import stats as st

% matplotlib inline

UsageError: Line magic function `%` not found.


In [21]:
# загрузим данные
data = pd.read_csv('../data/games.csv')
# дадим датафрейму имя для дальнейшего удобства использования
data.name = 'games'
# преобразуем названия колонок в нижний регистр
data.columns = [x.lower() for x in data.columns.to_list()]
# выведем датафрейм на экран
data.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


Посмотрим на пропуски в данных

In [20]:
def get_missing_values(data: pd.DataFrame) -> None:
    """
    Выводит данные о пропусках в колонках по датафрейму.
    Не изменяет данные внутри датафрейма.

    :param data: pd.DataFrame
    :return: None
    """
    # получаем имена колонок датафрейма
    columns = data.columns.to_list()
    data_len = len(data)
    # объявляем счетчик
    counter = -1
    display('='*60)
    # если есть пропуски в данных - выводим информацию о пропусках по колонкам
    if sum(data.isnull().sum()) > 0:
        display(f'Количество записей в датафрейме {data.name}: {data_len}')
        display(f'В датафрейме {data.name} имеются следующие пропуски:')
        for i in data.isnull().sum():
            counter += 1
            if i > 0:
                display(f'  - в колонке {columns[counter]}: {i} пропусков, это {i/data_len:0.2%} об общего объема данных')
    else:
        display(f'Отлично, в датафрейме {data.name} отсутствуют пропуски.')

# посмотрим на пропуски в данных
get_missing_values(data)

'============================================================'

'Количество записей в датафрейме games: 16715'

'В датафрейме games имеются следующие пропуски:'

'  - в колонке name: 2 пропусков, это 0.01% об общего объема данных'

'  - в колонке year_of_release: 269 пропусков, это 1.61% об общего объема данных'

'  - в колонке genre: 2 пропусков, это 0.01% об общего объема данных'

'  - в колонке critic_score: 8578 пропусков, это 51.32% об общего объема данных'

'  - в колонке user_score: 6701 пропусков, это 40.09% об общего объема данных'

'  - в колонке rating: 6766 пропусков, это 40.48% об общего объема данных'

Преобразуем типы данных

In [46]:
def auto_change_dtypes(data: pd.DataFrame) -> None:
    """
    Автоматически определяет тип столбца, и изменяет его в соответствии с хранимыми значениями.
    Функция не возвращает новый датафрейм, а изменяет переданный в качестве аргумента.
    Функция заточена под данные конкретного проекта.
    Функция поддерживает автоматическое преобразование следующих типов и форматов данных:
     - int64
     - float64
     - str: если в названии колонки есть date и формат даты %Y-%m-%d,
            то переводит в формат pandas datetime, иначе - переводит строковые данные в нижний регистр

    Пример преобразования:
    data[column] является int64 и содержит значения в диапазоне от 0 до 100 - будет преобразован в int8
    data[date_column] является object и содержит в имени колонки date - будет преобразован в datetime

    :param data: pd.DataFrame
    :return: None
    """
    # получаем количество используемой датафреймом памяти
    memory_usage_before_change_dtypes = data.memory_usage(index=False, deep=True).sum()
    # получаем описание датафрейма
    describe = data.describe()
    # получаем названия колонок
    columns = data.columns.to_list()
    # получаем типы данных
    dtypes = data.dtypes
    # количество типов данных
    indexes = len(dtypes)
    # создаем 2 словаря для int и float, содержащие в качестве ключей типы данных, а значений - список из min и max значений этих типов данных
    correct_int_dtypes = {'int8': [-2**7, 2**7-1], 'int16': [-2**15, 2**15-1], 'int32': [-2**31, 2**31-1]}
    correct_float_dtypes = {'float16': [-2.0**16, 2.0**16-1], 'float32': [-2.0**31, 2.0**31-1]}

    display(f'{"="*30} Работаем с датафреймом {data.name} {"="*30}')
    # пробегаем по индексам колонок датафрейма и типам данных колонок
    for index, dtype in zip(range(0, indexes), dtypes):

        # если тип int64, меняем на тип, соответствующий значениям колонок
        if dtype == np.int64:
            for key, value in zip(correct_int_dtypes.keys(), correct_int_dtypes.values()):
                if not describe[columns[index]]['min'] <= value[0] and not describe[columns[index]]['max'] >= value[1]:
                    display(f'Изменяем тип колонки {columns[index]} датафрейма {data.name} с {dtype} на {key}')
                    data[columns[index]] = data[columns[index]].astype(key)
                    break

        # если тип float64, меняем на тип, соответствующий значениям колонок
        elif dtype == np.float64:
            for key, value in zip(correct_float_dtypes.keys(), correct_float_dtypes.values()):
                if not describe[columns[index]]['min'] <= value[0] and not describe[columns[index]]['max'] >= value[1]:
                    display(f'Изменяем тип колонки {columns[index]} датафрейма {data.name} с {dtype} на {key}')
                    data[columns[index]] = data[columns[index]].astype(key)
                    break

        # если тип object и колонка содержит в названии 'date' - меняем на datetime
        elif dtype == object:
            if 'date' in columns[index]:
                display(f'Изменяем тип колонки {columns[index]} датафрейма {data.name} с {dtype} на datetime')
                data[columns[index]] = pd.to_datetime(data[columns[index]], format='%Y-%m-%d')
            # иначе приводим данные к нижнему регистру, пропуская значения nan (float) в колонке
            else:
                data[columns[index]] = data[columns[index]].apply(lambda s: s if type(s) == float else s.lower())

    # количество памяти, используемое датафреймом оптимизации типов данных
    memory_usage_after_change_dtypes = data.memory_usage(index=False, deep=True).sum()
    bytes_in_mb = 2**23
    display(f'Использование памяти датафрейма до сжатия: {(memory_usage_before_change_dtypes / bytes_in_mb):.2f} мб.')
    display(f'Использование памяти датафрейма после сжатия: {(memory_usage_after_change_dtypes / bytes_in_mb):.2f} мб.')
    display(f'Сжато: {((memory_usage_before_change_dtypes - memory_usage_after_change_dtypes) / bytes_in_mb):.2f} мб.')

auto_change_dtypes(data)

'============================== Работаем с датафреймом games =============================='

'Использование памяти датафрейма до сжатия: 0.62 мб.'

'Использование памяти датафрейма после сжатия: 0.62 мб.'

'Сжато: 0.00 мб.'

In [47]:
data.head(40)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,wii sports,wii,2006.0,sports,41.375000,28.953125,3.769531,8.453125,76.0,8,e
1,super mario bros.,nes,1985.0,platform,29.078125,3.580078,6.808594,0.770020,NaN,NaN,NaN
2,mario kart wii,wii,2008.0,racing,15.679688,12.757812,3.789062,3.289062,82.0,8.3,e
3,wii sports resort,wii,2009.0,sports,15.609375,10.929688,3.279297,2.949219,80.0,8,e
4,pokemon red/pokemon blue,gb,1996.0,role-playing,11.273438,8.890625,10.218750,1.000000,NaN,NaN,NaN
5,tetris,gb,1989.0,puzzle,23.203125,2.259766,4.218750,0.580078,NaN,NaN,NaN
6,new super mario bros.,ds,2006.0,platform,11.281250,9.140625,6.500000,2.880859,89.0,8.5,e
7,wii play,wii,2006.0,misc,13.960938,9.179688,2.929688,2.839844,58.0,6.6,e
8,new super mario bros. wii,wii,2009.0,platform,14.437500,6.941406,4.699219,2.240234,87.0,8.4,e
9,duck hunt,nes,1984.0,shooter,26.937500,0.629883,0.280029,0.469971,NaN,NaN,NaN
